# Automated UAS SkyWatch Standard Format Remark Functions & Script

## Origin Location: FAA Airport Identifier
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 19, 2021

***
***How to Use:***

`To obtain the Standard Format Remarks along with complete UAS Location information in the form of latitudes and longitudes, call: uas_lat_long('file_path')`

`This function will output the dataframe containing the complete UAS location information for Standard Format Remarks referencing an FAA Airport Identifier.`
***

## Import Statements

In [18]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

## Read in Files

In [2]:
# Read in Incidents_Cleaned_Standard.csv (contains 5191 records that are of 'Standard Format')
faa_standard = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard.csv')

# REMARK MUST REFERENCE AN FAA AIRPORT IDENTIFIER

## uas_loc_airportFAA()

Create a Function that takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses Regular Expressions to extract the portion of the REMARK that reads 'XX NM direction [of] XXX' from the remark where XXX references an FAA AIRPORT IDENTIFIER

In [19]:
# Create Function FOR STANDARD FORMAT REMARKS that:
# A. grabs REMARKS field from input dataframe/file.csv containing STANDARD FORMAT REMARKS
# B. Uses Regular Expressions to extract 'XX NM direction [of] XXX' from the remark

def uas_loc_airportFAA (file):
    '''
    This function takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses
    regular expressions to extract the portion of the remark that reads 'XX NM direction [of] XXX'.
    
    This function also exports the following .csv files:
    A) StandardRemark_UAS_Location_General_NULL -- Contains all UAS Locations from Standard Format Remarks (INCLUDING NULL VALUES)
    B) StandardRemark_UAS_Location_General_nonNULL -- Contains all UAS Locations from Standard Format Remarks (NOT INCLUDING NULL VALUES)
    
    @param file: the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    
    @output df1 : a dataframe containing the Standard Format REMARK as well as the UAS Location portion of the remark
                that reads 'XX NM direction [of] XXX' as well as the associated Airport information linked with the 
                Airport's FAA Identifier
    '''
    # Read in .csv file provided by user
    faa_standard = pd.read_csv(file)
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = faa_standard['REMARKS']

    # regular expression for any heading/direction
    headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?NM* [N|S|E|W|NW|NE|SW|SE|SSE|SSW|SNE|SNW|NNE|NSE|NNE|NNW|WSW|WNW|WSE|WNE|ENE|ESE|ESW|ENW]*\s?of?\s?[A-Z][A-Z][A-Z][A-Z]?'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        head_dir = re.findall(headir_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(head_dir)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    remark_uas_loc_df = pd.DataFrame()
    remark_uas_loc_df['REMARKS'] = remark
    remark_uas_loc_df['UAS Location'] = uas_loc
    
    # Export final dataframe (INCLUDING NULL UAS LOCATIONS) to .csv file
    remark_uas_loc_df.to_csv('StandardRemark_UAS_Location_IDENT_NULL.csv', index = False)
    
    # Get list of UAS locations from the above dataframe
    uas_loc = remark_uas_loc_df['UAS Location'] 

    # If the regular expressions did not hit on any location information, pass it UNKN for now
    for i in range(len(remark_uas_loc_df)):
        if len(uas_loc[i]) == 0:
            uas_loc[i] = 'UNKN'
    
    uas_loc_nonNull = uas_loc[uas_loc != 'UNKN'].to_list()
    uas_airport = []

    for i in range(len(uas_loc_nonNull)):
        airport = uas_loc_nonNull[i][0].split(' ')[-1]

        if len(airport) <= 4:
            uas_airport.append(airport)
        else:
            trim_air = airport[-3:]
            uas_airport.append(trim_air) # we perform this trim to account for a lack of space, for example: 'ofCLE'
    
    uas_airport_df = pd.DataFrame()
    uas_airport_df['IDENT'] = uas_airport
    
    # Create Dataframe of Standard Format Remarks with NON NULL UAS Locations
    remark_uas_loc_nn = remark_uas_loc_df[remark_uas_loc_df['UAS Location'] != 'UNKN'].reset_index()
    
    # Export final dataframe (EXCLUDING NULL UAS LOCATIONS) to .csv file
    remark_uas_loc_nn.to_csv('StandardRemark_UAS_Location_IDENT_nonNULL.csv', index = False)
    
    # JOIN DATASETS TO HAVE ALL LOCATION INFORMATION FROM REMARK

    # DATAFRAMES USED:
    # airportsC = airports_cleaned.csv (the cleaned airports dataset)
    # uas_airport_df = dataframe of FAA Airport Identifiers found in the 3033 Standard Formats hit on by Regular Expressions
    # full_loc = dataframe containing the UAS sighting remark from SkyWatch as well as the UAS location information extracted

    uas_air_loc= pd.merge(uas_airport_df, airportsC, on='IDENT', how='left')
    full_loc = pd.DataFrame()
    full_loc['REMARKS'] = remark_uas_loc_nn['REMARKS']
    full_loc['UAS_LOC'] = remark_uas_loc_nn['UAS Location']

    df1 = pd.concat([full_loc, uas_air_loc], axis = 1) #dataframe with final UAS Location information READY FOR CALCULATIONS
                                                                # from the 3,033 records extracted from the Standard Format Remarks
    
    df1 = df1.dropna().reset_index() # DROP ALL values that do not reference an airport using FAA Identifier
    
    return df1

In [23]:
uas_loc_airportFAA('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airport245.csv')

,index,REMARKS,UAS_LOC,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,0,Aircraft observed a orange uas off the left si...,[4 NM S of PTK],PTK,AAB633FC-88BB-495B-8628-761D29FC8C46,Oakland County Intl,42.665636,-83.420506,KPTK
1,1,Aircraft observed a white rotorcraft uas off t...,[3 NM W of OPF],OPF,039B278A-17B9-4F46-AA21-329CC7ED2D2C,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF
2,2,Aircraft observed a uas off the left side whil...,[1 NM NE of ADS],ADS,3646423B-964B-466A-9CE9-3CB43457F772,Addison,32.968556,-96.836444,KADS
3,3,"Aircraft observed a medium size, silver, singl...",[8.5 NM NE of ADS],ADS,3646423B-964B-466A-9CE9-3CB43457F772,Addison,32.968556,-96.836444,KADS
4,4,Aircraft observed a white uas off the right si...,[8 NM NW of DEN],DEN,0E6F2D50-AF47-445C-BE2A-B163C1F6EB04,Denver Intl,39.861667,-104.673167,KDEN
...,...,...,...,...,...,...,...,...,...
60,62,Aircraft observed a black and yellow single ro...,[4 NM W of LGB],LGB,B4B5BE1A-ECEC-4B67-BFA4-4D46C313674C,Long Beach (Daugherty Fld),33.817930,-118.151891,KLGB
61,63,Aircraft observed a group of quadcopter uas of...,[14 NM NW of PHX],PHX,639E89B2-F2AE-4589-BC8E-2694490CE138,Phoenix Sky Harbor Intl,33.434278,-112.011583,KPHX
62,64,Aircraft observed a Quad-rotor uas off the rig...,[7 NM SE of SRQ],SRQ,84E11735-61D9-4966-8E47-6B2E692C061A,Sarasota/Bradenton Intl,27.395444,-82.554389,KSRQ
63,65,Aircraft observed a uas 400 feet off the left ...,[7 NM E of SMO],SMO,FE544138-79F8-4437-AC45-7F6A688B86B9,Santa Monica Muni,34.015822,-118.451306,KSMO


## bearing_dist_FAAIdent()

Create a Function that takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses the uas_loc_airportFAA() function to split UAS Locations extracted from Remarks into their origin identifier, distance (in both NM and Kilometers) and bearing (as an abbreviation and in degrees).

In [20]:
def bearing_dist_FAAIdent(file):
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the uas_loc_airportFAA() function to split the UAS Location within the remark into the distance (in both NM and 
    kilometers), bearing abbreviation and degrees, and associated FAA airport Identifier.
    
    @param file : the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    @output new_df_validBearing: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance 
                                    (in NM) from the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks with invalid bearing information is 
                    exported into the StandardRemark_UAS_Location_Converted_allBearing.csv file 
                    
                *** it should be noted that the dataset that only contains remarks with VALID bearing information is
                    exported into the StandardRemark_UAS_Location_General_validBearing.csv file
    '''
    # Call uas_loc_airportFAA() function created above
    df1 = uas_loc_airportFAA(file)
    
    # Create Dictionary of Bearings and Respective Degrees
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    # Extract Distances and Bearings from UAS_LOC that were pulled from Remarks
    # with a Standard Format in SkyWatch
    uas_loc = df1['UAS_LOC']
    distances = []
    bearings = []

    for i in range(len(uas_loc)):
        split = uas_loc[i][0].split(' ')[0:3]

        if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
            distance_nm = split[0][0:len(split[0]) - 2]
            bearing_abrev = split[1].replace('of', '')

        else: 
            distance_nm = split[0]
            bearing_abrev = split[2].replace('of', '')

        distances.append(float(distance_nm))
        bearings.append(bearing_abrev)
    
    # Create DataFrame to store UAS Location Information
    new_df = pd.DataFrame(df1['REMARKS'])
    new_df['UAS_LOC'] = df1['UAS_LOC']
    new_df['IDENT'] = df1['IDENT']
    new_df['Distance_NM'] = distances
    new_df['Bearing'] = bearings
    
    # Convert all distance from NM to kilometers and all bearing abbreviations to their associated degrees
    dist_kilo = []

    for i in range(len(new_df)):
        distanceKilo = new_df['Distance_NM'][i] * 1.852 # converting NM to kilometers
        dist_kilo.append(distanceKilo)

    bearDegree = pd.DataFrame(new_df['Bearing'])
    bearDegree = bearDegree.replace({'Bearing': bearing_deg})
    
    for i in range(len(bearDegree)):
        if bearDegree['Bearing'][i] == '':
            bearDegree.replace('', np.nan, inplace = True)

    new_df['Distance_Kilometers'] = dist_kilo
    new_df['Bearing_Degrees'] = bearDegree
    
    new_df['Airport_Latitude'] = df1['LATITUDE']
    new_df['Airport_Longitude'] = df1['LONGITUDE']
    
    # Export final new_df to .csv 
    new_df.to_csv('StandardRemark_UAS_Location_Converted_IDENTallBearing.csv', index = False)
    
    # Create Dataframe of Standard Format Remarks without NULL BEARING INFO
    new_df_validBearing = new_df.dropna().reset_index()
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    new_df_validBearing.to_csv('StandardRemark_UAS_Location_General_IDENTvalidBearing.csv', index = False)
    
    return new_df_validBearing

In [15]:
bearing_dist_FAAIdent('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard_NoDuplicates.csv')

,index,REMARKS,UAS_LOC,IDENT,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,0,Aircraft observed alarge orange UAS with flash...,[3 NM NNW of CLE],CLE,3.0,NNW,5.556,338.0,41.409407,-81.854691
1,1,"Aircraft observed a UAS while at 5,000 feet 11...",[11 NM SSE of SLC],SLC,11.0,SSE,20.372,158.0,40.788388,-111.977773
2,2,Aircraft observed a multi-colored UAS off the ...,[7NM E of SAN],SAN,7.0,E,12.964,90.0,32.733556,-117.189667
3,3,"Aircraft observed a triangular shaped, grey an...",[23NM SW of ORL],ORL,23.0,SW,42.596,225.0,28.545462,-81.332930
4,4,Aircraft observed a red quad copter UAS while ...,[4NM SSE of MLB],MLB,4.0,SSE,7.408,158.0,28.102750,-80.645250
...,...,...,...,...,...,...,...,...,...,...
2799,2805,Aircraft reported a medium sized quad-copter U...,[3 NM SW of SAT],SAT,3.0,SW,5.556,225.0,29.533958,-98.469057
2800,2806,Aircraft reported a quad-copter UAS at 12 O'cl...,[10 NM SE of ADS],ADS,10.0,SE,18.520,135.0,32.968556,-96.836444
2801,2807,Aircraft reported a UAS 3 NM N of MMU while E ...,[3 NM N of MMU],MMU,3.0,N,5.556,0.0,40.799338,-74.414889
2802,2808,Aircraft reported a UAS sensor hit while N bou...,[21 NM NW of BXK],BXK,21.0,NW,38.892,315.0,33.420417,-112.686181


## uas_FAA_lat_long()

Calculates the latitude and longitude for the sighted UAS using the geopy library (***WHEN THE UAS SIGHTING REFERENCES AN FAA AIRPORT IDENTIFIER***)

In [21]:
def uas_FAA_lat_long(file):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param file : the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    # Call bearing_dist_originIdent() function created above
    new_df = bearing_dist_FAAIdent(file)
    
    uas_lat = []
    uas_long = []


    for i in range(len(new_df)):
        lat_airport = pd.to_numeric(new_df['Airport_Latitude'][i])
        long_airport = pd.to_numeric(new_df['Airport_Longitude'][i])
        b = pd.to_numeric(new_df['Bearing_Degrees'][i])
        d = pd.to_numeric(new_df['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2, = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
    
    # Append UAS Lat/Long information to DataFrame
    new_df['UAS_Latitude'] = uas_lat
    new_df['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    new_df.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return new_df

In [22]:
uas_FAA_lat_long('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airport245.csv')

,index,REMARKS,UAS_LOC,IDENT,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude,UAS_Latitude,UAS_Longitude
0,0,Aircraft observed a orange uas off the left si...,[4 NM S of PTK],PTK,4.0,S,7.408,180,42.665636,-83.420506,42.598949,-83.420506
1,1,Aircraft observed a white rotorcraft uas off t...,[3 NM W of OPF],OPF,3.0,W,5.556,270,25.907417,-80.278222,25.907406,-80.333674
2,2,Aircraft observed a uas off the left side whil...,[1 NM NE of ADS],ADS,1.0,NE,1.852,45,32.968556,-96.836444,32.980363,-96.822435
3,3,"Aircraft observed a medium size, silver, singl...",[8.5 NM NE of ADS],ADS,8.5,NE,15.742,45,32.968556,-96.836444,33.068867,-96.717241
4,4,Aircraft observed a white uas off the right si...,[8 NM NW of DEN],DEN,8.0,NW,14.816,315,39.861667,-104.673167,39.955957,-104.795772
...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,Aircraft observed a black and yellow single ro...,[4 NM W of LGB],LGB,4.0,W,7.408,270,33.817930,-118.151891,33.817904,-118.231907
61,61,Aircraft observed a group of quadcopter uas of...,[14 NM NW of PHX],PHX,14.0,NW,25.928,315,33.434278,-112.011583,33.599419,-112.209112
62,62,Aircraft observed a Quad-rotor uas off the rig...,[7 NM SE of SRQ],SRQ,7.0,SE,12.964,135,27.395444,-82.554389,27.312686,-82.461774
63,63,Aircraft observed a uas 400 feet off the left ...,[7 NM E of SMO],SMO,7.0,E,12.964,90,34.015822,-118.451306,34.015742,-118.310954
